---

# Main Script to Run

---

## Steps

1. Open a Paper on Zotero
2. Copy the paper into a text file named "paper.txt". Save this file into a folder named after the author. (ex. Doe et al., 2015)
3. Create annotations, extract them into a text file, and save them as "notes.txt"
4. Drag & drop the folders into the "_inbox" folder, in the same parent folder where all the paper folders are stored. The folders in the _inbox folder will be the ones that the script finds and acts on.
5. Run the Script 
6. A set of Anki flashcards will be generated

## complete example

In [1]:
import os
import requests
from dotenv import load_dotenv

# Load environment variables from .env
load_dotenv()

# Retrieve your OpenAI API key from environment variables
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

# Define the path to your _inbox folder
inbox_path = r"D:\OneDrive\_Carnegie Mellon (CMU)\60 Academic\63 Literature Review\63.002 Literature Review Exports from Zotero\_inbox"


# Update these paths as needed
INBOX_PATH = inbox_path  # Ensure this variable is defined appropriately
ANKICONNECT_URL = "http://127.0.0.1:8765"
OPENAI_URL = "https://api.openai.com/v1/chat/completions"

# Parent deck name
PARENT_DECK = "CMU.63 Literature Review::63.003 Transparent Electrodes"
# PARENT_DECK = "CMU.63 Literature Review::63.002 Nano-Scale Bioelectronics"

In [2]:


def create_deck(deck_name):
    """
    Creates a deck in Anki using AnkiConnect. If the deck already exists,
    this action will not have any adverse effect.
    """
    payload = {
        "action": "createDeck",
        "version": 6,
        "params": {
            "deck": deck_name
        }
    }
    response = requests.post(ANKICONNECT_URL, json=payload)
    return response.json()

def add_card_to_anki(deck_name, front, back):
    """
    Ensures the deck exists, then adds a Basic card to it.
    """
    create_deck(deck_name)
    
    payload = {
        "action": "addNotes",
        "version": 6,
        "params": {
            "notes": [
                {
                    "deckName": deck_name,
                    "modelName": "Basic",
                    "fields": {
                        "Front": front,
                        "Back": back
                    },
                    "tags": ["paper", "notecard"]
                }
            ]
        }
    }
    response = requests.post(ANKICONNECT_URL, json=payload)
    return response.json()

def generate_notecards(notes_text):
    """
    Sends the content of notes.txt to the OpenAI Chat Completions endpoint
    and returns the raw text containing Q&A pairs formatted as notecards.
    """
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {OPENAI_API_KEY}"
    }
    prompt_messages = [
        {
            "role": "system",
            "content": (
                "You are given annotations from a research paper. "
                "Create concise question-answer pairs as notecards. "
                "Each notecard should start with 'Q:' for the question and 'A:' for the answer."
                "Make sure to create enough cards to cover all the content in the notes file, "
                "but keep the total card count no greater than 10-15 cards. "
            )
        },
        {
            "role": "user",
            "content": notes_text
        }
    ]
    payload = {
        "model": "gpt-3.5-turbo",
        "messages": prompt_messages,
        "temperature": 0.7
    }
    response = requests.post(OPENAI_URL, headers=headers, json=payload)
    data = response.json()
    return data["choices"][0]["message"]["content"]

def parse_q_and_a(generated_text):
    """
    Parses the generated text into a list of (question, answer) tuples.
    Expects the format:
      Q: Question text
      A: Answer text
    """
    lines = generated_text.splitlines()
    flashcards = []
    question = ""
    answer = ""
    for line in lines:
        line = line.strip()
        if line.startswith("Q:"):
            if question and answer:
                flashcards.append((question, answer))
                question, answer = "", ""
            question = line[2:].strip()
        elif line.startswith("A:"):
            answer = line[2:].strip()
    if question and answer:
        flashcards.append((question, answer))
    return flashcards

def main():
    if not os.path.exists(INBOX_PATH):
        print(f"Inbox path does not exist: {INBOX_PATH}")
        return

    # List all folders in the _inbox
    folder_names = [
        folder for folder in os.listdir(INBOX_PATH)
        if os.path.isdir(os.path.join(INBOX_PATH, folder))
    ]

    for folder in folder_names:
        folder_path = os.path.join(INBOX_PATH, folder)
        notes_file = os.path.join(folder_path, "notes.txt")
        paper_file = os.path.join(folder_path, "paper.txt")

        if not (os.path.exists(notes_file) and os.path.exists(paper_file)):
            print(f"Missing notes.txt or paper.txt in folder: {folder}")
            continue

        with open(notes_file, "r", encoding="utf-8") as nf:
            notes_text = nf.read()

        # Generate notecards text via OpenAI
        raw_notecards_text = generate_notecards(notes_text)
        # Parse the raw text into Q&A pairs
        notecards = parse_q_and_a(raw_notecards_text)

        # Create the full deck name using the parent deck and the folder name
        full_deck_name = f"{PARENT_DECK}::{folder}"

        for question, answer in notecards:
            result = add_card_to_anki(full_deck_name, question, answer)
            print(f"Added notecard to deck '{full_deck_name}': {result}")

if __name__ == "__main__":
    main()


Added notecard to deck 'CMU.63 Literature Review::63.003 Transparent Electrodes::Khaldi et al., 2016': {'result': [1741199416658], 'error': None}
Added notecard to deck 'CMU.63 Literature Review::63.003 Transparent Electrodes::Khaldi et al., 2016': {'result': [1741199416721], 'error': None}
Added notecard to deck 'CMU.63 Literature Review::63.003 Transparent Electrodes::Khaldi et al., 2016': {'result': [1741199416784], 'error': None}
Added notecard to deck 'CMU.63 Literature Review::63.003 Transparent Electrodes::Khodagholy et al., 2011': {'result': [1741199419837], 'error': None}
Added notecard to deck 'CMU.63 Literature Review::63.003 Transparent Electrodes::Khodagholy et al., 2011': {'result': [1741199419898], 'error': None}
Added notecard to deck 'CMU.63 Literature Review::63.003 Transparent Electrodes::Khodagholy et al., 2011': {'result': [1741199419959], 'error': None}
Added notecard to deck 'CMU.63 Literature Review::63.003 Transparent Electrodes::Khodagholy et al., 2011': {'res